In [1]:
from evaluate import load
comet_metric = load('comet')
source = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
hypothesis = ["The fire could be stopped", "Schools and kindergartens were open"]
reference = ["They were able to control the fire.", "Schools and kindergartens opened"]
comet_score = comet_metric.compute(predictions=hypothesis, references=reference, sources=source)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
Encoder model frozen.
/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/ ...
GPU available: True (cuda), used: True
TPU av

In [1]:
from evaluate import load
comet_metric = load('comet')
sacrebleu_metric = load('sacrebleu')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
Encoder model frozen.
/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
import numpy as np
from scipy.special import softmax, kl_div

def entropy(p):
    return -np.sum(p * np.log(p + 1e-10), axis=-1)  # Compute along vocab axis

def js_divergence(logits1, logits2):
    p1 = softmax(logits1, axis=-1)  # Apply softmax along vocab axis
    p2 = softmax(logits2, axis=-1)
    m = 0.5 * (p1 + p2)
    
    kl1 = np.sum(kl_div(p1, m), axis=1)  # Compute KL-div along vocab axis
    kl2 = np.sum(kl_div(p2, m), axis=1)
    print(kl1)
    
    return 0.5 * (kl1 + kl2)  # JSD for each batch element

# Example usage with batch dimension:
logits1 = np.array([[1.0, 2.0, 3.0], [2.0, 1.0, 0.5]])  # Shape (B, V)
logits2 = np.array([[-1.5, -10.5, -3.5], [5.5, 6.5, 1.0]])

p1 = softmax(logits1, axis=-1)
p2 = softmax(logits2, axis=-1)

entropy_diff = entropy(p1) - entropy(p2)  # Shape (B,)
jsd = js_divergence(logits1, logits2)  # Shape (B,)

print(f"Entropy Difference (batch-wise): {entropy_diff}")
print(f"Jensen-Shannon Divergence (batch-wise): {jsd}")

[0.3693169 0.1376967]
Entropy Difference (batch-wise): [0.46600076 0.30519081]
Jensen-Shannon Divergence (batch-wise): [0.37566115 0.14739189]


In [44]:
in_file = 'outputs/qwen-instruct/generated_texts_en-hi_conditioned_qwen-7b-instruct_random-5_activated_multiplier_5.txt'
src_file = '../datasets/calcs_english-hinglish/dev_en.txt'
ref_file = '../datasets/calcs_english-hinglish/dev_hi_og_new.txt'
ref_file_2 = '../datasets/calcs_english-hinglish/dev_natural_cmix.txt'

import numpy as np
from tqdm.contrib import tzip

def load_txt_file(filepath):
    lines = []
    with open(filepath, 'r') as f:
        for line in f.readlines():
            lines.append(line.strip())
    return lines

def load_txt_file_with_sep(filepath,sep='|'):
    lines = []
    with open(filepath, 'r') as f:
        for idx, line in enumerate(f.readlines()):
            if idx == 0:
                continue
            text = line.split(sep)[0]
            lines.append(text.strip())
    return lines

source_texts = load_txt_file(src_file)
ref_texts = load_txt_file(ref_file)
pred_texts = load_txt_file_with_sep(in_file)
ref_texts_2 = load_txt_file(ref_file_2)
pred_texts = pred_texts[:len(ref_texts)]
bleu_score = sacrebleu_metric.compute(references=ref_texts, predictions=pred_texts)
bleu_score_2 = sacrebleu_metric.compute(references=ref_texts_2, predictions=pred_texts)
comet_score = comet_metric.compute(predictions=pred_texts, references=ref_texts, sources=source_texts)
comet_score_2 = comet_metric.compute(predictions=pred_texts, references=ref_texts_2, sources=source_texts)
print(f"BLEU Score: {max(bleu_score['score'],bleu_score_2['score'])}")
print(f"Comet Score: {max(comet_score['mean_score'], comet_score_2['mean_score'])}")



    


/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 H

BLEU Score: 6.408658763227477
Comet Score: 0.5419043534437309


In [6]:
comet_score = comet_metric.compute(predictions=['namaste'], references=['namaste'], sources=['hello'])
#print(f"BLEU Score: {bleu_score['score']}")
print(f"Comet Score: {comet_score['mean_score']}")

/home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mahardika.ihsani/.conda/envs/cmix_neurons/lib/ ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Comet Score: 0.9942197203636169


In [21]:
sacrebleu_metric = load('sacrebleu')


KeyboardInterrupt: 

In [22]:
source = ["Dem Feuer konnte Einhalt geboten werden", "Schulen und Kindergärten wurden eröffnet."]
predictions = ["The fire could be stopped", "Schools and kindergartens were open"]
references = ["They were able to control the fire.", "Schools and kindergartens opened"]
bleu_score = sacrebleu_metric.compute(references=references, predictions=predictions)

In [23]:
bleu_score

{'score': 17.491650626361256,
 'counts': [4, 2, 1, 0],
 'totals': [10, 8, 6, 4],
 'precisions': [40.0, 25.0, 16.666666666666668, 12.5],
 'bp': 0.8187307530779819,
 'sys_len': 10,
 'ref_len': 12}

In [4]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)
model.predict("Hello, world!")


model.predict("Mujhe", k=5)


(('__label__eng_Latn',
  '__label__deu_Latn',
  '__label__slk_Latn',
  '__label__ces_Latn',
  '__label__kor_Hang'),
 array([0.89432603, 0.08194095, 0.02017043, 0.00129148, 0.00111642]))

(('__label__eng_Latn',
  '__label__est_Latn',
  '__label__lit_Latn',
  '__label__ind_Latn',
  '__label__vie_Latn',
  '__label__nld_Latn',
  '__label__afr_Latn',
  '__label__nno_Latn',
  '__label__bod_Tibt',
  '__label__swe_Latn'),
 array([0.54486501, 0.29932994, 0.06832899, 0.02347875, 0.01895998,
        0.0185504 , 0.00834634, 0.00625697, 0.00249793, 0.00242909]))